In [1]:
import timeit

import numpy as np
import sympy as sym

In [2]:
# helper functions to be moved later to own module
def create_variables_and_differentials(variables_string, parameter_var_string=None):
    q = sym.symbols(variables_string) # tuple
    if parameter_var_string is None:
        q = sym.Matrix(q)
        dq = sym.Matrix(
            [sym.symbols(f'd{q[i].name}') for i in range(len(q))]
        )
    else:
        if parameter_var_string == '':
            raise ValueError(
                f'{parameter_var_string} -- parameter_var_string, cannot be an empty string, use None!'
            )
        param_var = sym.symbols(parameter_var_string)
        q = sym.Matrix(
            [sym.Function(e.name)(param_var) for e in q]
        )
        dq = sym.diff(q, param_var)
    return q, dq

In [3]:
def line_element_to_metric_tensor(ds_sqrd, dq):
    n = len(dq)
    dq_dq_permutations = sym.tensorproduct(dq, dq).reshape(n ** 2, 1)
    # must expand so coeff method will work properly!
    g = sym.Matrix(
        [sym.expand(ds_sqrd).coeff(e[0], 1) for e in dq_dq_permutations]
    )
    g = g.reshape(n, n)
    diag_g = sym.Matrix(np.diag(np.diag(g)))
    g = (g + diag_g) / 2
    return g


def metric_tensor_to_line_element(g, dq):
    return sym.expand(np.dot(np.dot(dq.T, g), dq).flatten()[0])


### test the two functions are inverses of each other
def test_line_elem_metric_inverses():
    bools = []
    
    # create variables and dq
    c, dt, dx, dy, dz = sym.symbols('c dt dx dy dz')
    dq = sym.Matrix([dt, dx, dy, dz])
    
    # forward direction
    line_element = sym.expand(-c ** 2 * dt ** 2 + dx ** 2 + dy ** 2 + dz ** 2 + 2 * dt * dz)
    g = line_element_to_metric_tensor(line_element, dq)
    bools.append(
        sym.Equality(line_element, metric_tensor_to_line_element(g, dq))
    )
    # backward direction, choosing a different metric tensor that is not symmetric
    g = sym.Matrix(
        [
            [-c ** 2, 0, 0, 2],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ]
    )
    g = (g + g.T) / 2 # make symmetric
    line_element = metric_tensor_to_line_element(g, dq)
    bools.append(
        sym.Equality(g, line_element_to_metric_tensor(line_element, dq))
    )
    return all(bools)


print(
    f'{test_line_elem_metric_inverses()} -- '
    'line element to metric tensor, and metric tensor to line element are inverse methods.'
)

True -- line element to metric tensor, and metric tensor to line element are inverse methods.


In [4]:
def symbolic_to_numpy_func(symbolic_expr_obj, symbolic_variables):
    """
    Converts a symbolic object like a symbol or Matrix into a numpy function
    that returns a numpy array of the same shape as that object.

    Parameters
    ----------
    symbolic_expr_obj : sympy expression
        A sympy expression, list of expressions, or matrix to be evaluated.
    symbolic_variables : list of sympy variables
        A variable or a list of variables, represented in the way that
        arguments will be passed to the function.
        Includes symbols, undefined functions, or matrix symbols.

    Returns
    -------
    function
        Function that will return a numpy array of the same shape as the 
        symbolic_expr_obj.

    """
    return sym.lambdify(symbolic_variables, symbolic_expr_obj, modules='numpy')


def symbolic_obj_subs(symbolic_expr_obj, symbolic_variables,
                      variable_constants):
    """
    Converts a symbolic object like a symbol or a matrix into a constant
    version of that object, where all symbolic_variables listed have been 
    substituted with thier respective variable_constants

    Parameters
    ----------
    symbolic_expr_obj : sympy expression
        A sympy expression, list of expressions, or matrix to be evaluated.
    symbolic_variables : list of sympy variables
        A variable or a list of variables, represented in the way that
        arguments will be passed to the function.
        Includes symbols, undefined functions, or matrix symbols.
    variable_constants : list of arrays likes
        The values that will be used to substitute symbolic_variables.

    Returns
    -------
    symbolic_const_obj : numpy array
        numpy array of the sampe shape as the symbolic_expr_obj, now with
        all the provided variables substituted with constants.

    """
    symbolic_variables = np.concatenate(symbolic_variables).flatten()
    variable_constants = np.concatenate(variable_constants)
    subs_map = {
        str(k):v 
        for k, v in zip(symbolic_variables, variable_constants)
    }
    if isinstance(symbolic_expr_obj, list):
        symbolic_const_obj = [o.subs(subs_map) for o in symbolic_expr_obj]
    else:
        symbolic_const_obj = symbolic_expr_obj.subs(subs_map)
    return symbolic_const_obj


def symbolic_const_matrix_to_numpy(A):
    return np.array(A).astype(float)

In [5]:
q, dq = create_variables_and_differentials('t, r, theta, phi', parameter_var_string=None)
dq

Matrix([
[    dt],
[    dr],
[dtheta],
[  dphi]])

In [6]:
q

Matrix([
[    t],
[    r],
[theta],
[  phi]])

In [7]:
# params = sym.Matrix(sym.symbols('M a'))
params = sym.Matrix([sym.symbols('M')])
### TODO when there is only one param need to change how this is constructed
params

Matrix([[M]])

In [8]:
# schwarzchild
line_element = (
    (1 - 2 * params[0] / q[1]) * dq[0] ** 2 
    - (1 - 2 * params[0] / q[1]) ** -1 * dq[1] ** 2 
    - q[1] ** 2 * (dq[2] ** 2 + sym.sin(q[2]) ** 2 * dq[3] ** 2)
)
# kerr
# rs = 2 * params[0]
# rho_sqrd = q[1] ** 2 + params[1] ** 2 * sym.cos(q[2]) ** 2
# delta = q[1] ** 2 - rs * q[1] + params[1] ** 2
# line_element = (
#     -(1 - ((rs * q[1]) / rho_sqrd)) * dq[0] ** 2
#     + rho_sqrd / delta * dq[1] ** 2
#     + rho_sqrd * dq[2] ** 2
#     + (q[1] ** 2 + params[1] ** 2 + ((rs * q[1] * params[1] ** 2) / rho_sqrd) * sym.sin(q[2]) ** 2) * dq[3] ** 2
#     - (2 * rs * q[1] * params[1] * sym.sin(q[2]) ** 2) / rho_sqrd * dq[0] * dq[3]
# )

line_element

-dr**2/(-2*M/r + 1) + dt**2*(-2*M/r + 1) - r**2*(dphi**2*sin(theta)**2 + dtheta**2)

In [9]:
g_sym = line_element_to_metric_tensor(line_element, dq)
g_sym = -g_sym
g_sym
### note if signature has changed, need to multiply by -1 to correct if so

Matrix([
[2*M/r - 1,              0,    0,                  0],
[        0, 1/(-2*M/r + 1),    0,                  0],
[        0,              0, r**2,                  0],
[        0,              0,    0, r**2*sin(theta)**2]])

In [10]:
# %timeit sym.Matrix(np.diag(np.diag(g))) # fastest to use numpy
# 97.6 µs ± 566 ns

In [11]:
# %timeit sym.diag(*np.diag(g))
# 152 µs ± 694 ns

In [12]:
# %timeit g.upper_triangular()
# 141 µs ± 400 ns

In [13]:
# %timeit (g + sym.Matrix(np.diag(np.diag(g)))) / 2
# 191 µs ± 1.09 µs

In [14]:
params_const = np.array([1,])
q0 = np.array([0, 40, np.pi / 2, 0])
# q0 = [0, 20, np.pi / 2, 0]

In [15]:
p0 = sym.Matrix([sym.symbols('p_0'), 0, 0, 3.83405])
# p0 = sym.Matrix([sym.symbols('p_0'), 0, 3.8, 3])
p0

Matrix([
[    p_0],
[      0],
[      0],
[3.83405]])

In [35]:
g_sym_inv = g_sym.inv()
g_sym_inv

Matrix([
[1/(2*M/r - 1),          0,       0,                      0],
[            0, -2*M/r + 1,       0,                      0],
[            0,          0, r**(-2),                      0],
[            0,          0,       0, 1/(r**2*sin(theta)**2)]])

In [36]:
g_sym_inv_0 = symbolic_obj_subs(g_sym_inv, [params, q], [params_const, q0])

g_sym_inv_0

Matrix([
[-1.05263157894737,    0,        0,        0],
[                0, 0.95,        0,        0],
[                0,    0, 0.000625,        0],
[                0,    0,        0, 0.000625]])

In [29]:
# %timeit np.array(g_sym_inv_0).astype(float)
# 62.3 µs ± 249 ns

In [30]:
# %timeit sym.lambdify((), g_sym_inv_0, modules='numpy')()
# 889 µs ± 8.49 µs

In [31]:
# %timeit np.dot(np.dot(p0.T, g_sym_inv_0), p0).flatten()[0] + 1
# 224 µs ± 1.03 µs

In [32]:
# %timeit sym.Mul(sym.Mul(p0.T, g_sym_inv_0), p0)[0] + 1
# 689 µs ± 992 ns

In [37]:
p_0_poly = np.dot(np.dot(p0.T, g_sym_inv_0), p0).flatten()[0] + 1  # add 1 for timelike
p_0_poly

1.00918746212656 - 1.05263157894737*p_0**2

In [38]:
sorted(sym.solve(p_0_poly, 'p_0'))

[-0.979146612627666, 0.979146612627666]

In [39]:
symbolic_obj_subs(params[0], [sym.Matrix([sym.symbols('M')])], [[1]])

1

In [40]:
symbolic_obj_subs(params, [params], [[1]])

Matrix([[1]])

In [41]:
symbolic_obj_subs(params, [params], [[1, 0]])

Matrix([[1]])

In [42]:
g_sym_inv_part_const = symbolic_obj_subs(
    -g_sym_inv,
    [params],
    [params_const]
)
g_sym_inv_part_const

Matrix([
[-1/(-1 + 2/r),        0,       0,                       0],
[            0, -1 + 2/r,       0,                       0],
[            0,        0, -1/r**2,                       0],
[            0,        0,       0, -1/(r**2*sin(theta)**2)]])

In [43]:
g_sym_inv_part_const_func = symbolic_to_numpy_func(
    g_sym_inv_part_const, 
    q
)
g_sym_inv_part_const_func

<function _lambdifygenerated(t, r, theta, phi)>

In [44]:
g_sym_inv_part_const_func(*q0)

array([[ 1.05263158e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -9.50000000e-01,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -6.25000000e-04,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.25000000e-04]])

In [45]:
g_sym_inv_part_const_func = symbolic_to_numpy_func(
    g_sym_inv_part_const, 
    [q]
)
g_sym_inv_part_const_func(q0)

array([[ 1.05263158e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -9.50000000e-01,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -6.25000000e-04,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.25000000e-04]])